<a href="https://colab.research.google.com/github/JotaPMendes/faculdade_cdia_2023_2026/blob/main/6_semestre/consultoria_ciencia_dados/projeto_final/tratamento_dados.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [ ]:
import pandas as pd
from google.colab import drive
import re, unicodedata
import numpy as np

In [ ]:
drive.mount('/content/MyDrive')

Drive already mounted at /content/MyDrive; to attempt to forcibly remount, call drive.mount("/content/MyDrive", force_remount=True).


# Carregando a base de Dados

In [ ]:
base = pd.read_excel('/content/MyDrive/MyDrive/faculdade_cdia_2023_2026/6-semestre/consultoria_ciencia_dados_6-semestre/projeto_final/base_puc.xlsx', header=None)

In [ ]:
base.head()

,0,1,2,3,4,5,6,7,8,9,...,118,119,120,121,122,123,124,125,126,127
0,NaN,NaN,NaN,NaN,Aula 1,NaN,NaN,NaN,NaN,Aula 2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,8-fev.-2025,NaN,NaN,NaN,NaN,15-fev.-2025,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Nome Planilha Feedback,Sala,Num,NOME COMPLETO,Pre-Class,P,Hw,CP,Bh,Pre-Class,...,e-mail Responsável,Data do Ultimo Login Conta CPM,Status da Conta CPM,Quantidade de reprovações no curso (4 reprovaç...,Status aluno semestre anterior,Aluno pode trancar matrícula ou manter trancam...,Situação aluno no semestre,"Se aluno não está mais frequentando as aulas, ...",Situação Final,Recomendação Conselho (Aprovado/Reprovado)
3,Four Corners A1 - 1S2025 - Capão Redondo - Fee...,Four Corners A1,1,Estudante 1 - Capão Redondo Four Corners A1,NaN,A,NaN,NaN,NaN,√,...,NaN,2025-05-06 00:39:06,Ativa,0,Aluno Novo,Sim,NaN,NaN,Reprovado,NaN
4,Four Corners A1 - 1S2025 - Capão Redondo - Fee...,Four Corners A1,2,Estudante 2 - Capão Redondo Four Corners A1,NaN,A,NaN,NaN,NaN,√,...,NaN,2025-07-06 01:14:04,Ativa,0,Aluno Novo,Sim,NaN,NaN,Aprovado,NaN


# Trocando os nomes das colunas de aula que estão merged

In [ ]:
linha_aulas = base.iloc[0]
linha_datas = base.iloc[1]
linha_metadados = base.iloc[2]

novas_colunas = []
novas_colunas.extend(base.columns[:4])

n_colunas_por_aula = 5
n_aulas = 14

for i in range(n_aulas):
    start_col = 4 + i * n_colunas_por_aula
    end_col = start_col + n_colunas_por_aula

    bloco_aulas = linha_aulas.iloc[start_col:end_col]
    bloco_datas = linha_datas.iloc[start_col:end_col]
    bloco_metadados = linha_metadados.iloc[start_col:end_col]

    aula_val = bloco_aulas.dropna().iloc[0] if not bloco_aulas.dropna().empty else f"Aula_{i+1}"
    data_val = bloco_datas.dropna().iloc[0] if not bloco_datas.dropna().empty else "Data_indefinida"

    bloco_metadados_filled = bloco_metadados.ffill().bfill()

    for meta_val in bloco_metadados_filled:
        meta_str = str(meta_val).strip() if pd.notna(meta_val) else "Sem_metadado"
        novas_colunas.append(f"{aula_val} - {data_val} - {meta_str}")

novas_colunas.extend(base.columns[4 + n_aulas * n_colunas_por_aula:])

if len(novas_colunas) != len(base.columns):
    print(f"❌ Erro: {len(novas_colunas)} novos nomes ≠ {len(base.columns)} originais.")
else:
    print("✅ Novos nomes de colunas criados com sucesso.")
    base.columns = novas_colunas


✅ Novos nomes de colunas criados com sucesso.


# Trocando os nomes das colunas de avaliação dos alunos

In [ ]:
def excel_col_to_pos(col_letters: str):
    s = col_letters.strip().upper()
    acc = 0
    for ch in s:
        if not ("A" <= ch <= "Z"):
            raise ValueError(f"Letra inválida: {col_letters}")
        acc = acc * 26 + (ord(ch) - ord('A') + 1)
    return acc - 1  # 0-based

mapa_colunas = {
    "pre_class": ["completed", "partial", "not_done", "total", "percent"],
    "presence": ["present", "absent", "half", "total", "percent"],
    "homework": ["completed", "partial", "not_done", "total", "percent"],
    "class_participation": ["positive", "neutral", "negative", "total", "percent"],
    "behavior": ["negative", "mediano", "positive", "percent"],
}

start_pos = excel_col_to_pos("CJ")
new_cols = list(base.columns)
n_cols = len(new_cols)

pos_atual = start_pos
for prefix, sufixos in mapa_colunas.items():
    for suf in sufixos:
        if pos_atual < n_cols:
            new_cols[pos_atual] = f"{prefix}_{suf}"
            pos_atual += 1
        else:
            break

base.columns = new_cols

print("✅ Colunas renomeadas de CJ em diante:")
for idx in range(start_pos - 2, min(pos_atual + 2, n_cols)):
    print(f"{idx:>3}: {base.columns[idx]}")


✅ Colunas renomeadas de CJ em diante:
 85: 85
 86: 86
 87: pre_class_completed
 88: pre_class_partial
 89: pre_class_not_done
 90: pre_class_total
 91: pre_class_percent
 92: presence_present
 93: presence_absent
 94: presence_half
 95: presence_total
 96: presence_percent
 97: homework_completed
 98: homework_partial
 99: homework_not_done
100: homework_total
101: homework_percent
102: class_participation_positive
103: class_participation_neutral
104: class_participation_negative
105: class_participation_total
106: class_participation_percent
107: behavior_negative
108: behavior_mediano
109: behavior_positive
110: behavior_percent
111: 111
112: 112


In [ ]:
base = base.iloc[2:]
base.head()

,0,1,2,3,Aula 1 - 8-fev.-2025 - Pre-Class,Aula 1 - 8-fev.-2025 - P,Aula 1 - 8-fev.-2025 - Hw,Aula 1 - 8-fev.-2025 - CP,Aula 1 - 8-fev.-2025 - Bh,Aula 2 - 15-fev.-2025 - Pre-Class,...,118,119,120,121,122,123,124,125,126,127
2,Nome Planilha Feedback,Sala,Num,NOME COMPLETO,Pre-Class,P,Hw,CP,Bh,Pre-Class,...,e-mail Responsável,Data do Ultimo Login Conta CPM,Status da Conta CPM,Quantidade de reprovações no curso (4 reprovaç...,Status aluno semestre anterior,Aluno pode trancar matrícula ou manter trancam...,Situação aluno no semestre,"Se aluno não está mais frequentando as aulas, ...",Situação Final,Recomendação Conselho (Aprovado/Reprovado)
3,Four Corners A1 - 1S2025 - Capão Redondo - Fee...,Four Corners A1,1,Estudante 1 - Capão Redondo Four Corners A1,NaN,A,NaN,NaN,NaN,√,...,NaN,2025-05-06 00:39:06,Ativa,0,Aluno Novo,Sim,NaN,NaN,Reprovado,NaN
4,Four Corners A1 - 1S2025 - Capão Redondo - Fee...,Four Corners A1,2,Estudante 2 - Capão Redondo Four Corners A1,NaN,A,NaN,NaN,NaN,√,...,NaN,2025-07-06 01:14:04,Ativa,0,Aluno Novo,Sim,NaN,NaN,Aprovado,NaN
5,Four Corners A1 - 1S2025 - Capão Redondo - Fee...,Four Corners A1,3,Estudante 3 - Capão Redondo Four Corners A1,NaN,A,NaN,NaN,NaN,N,...,NaN,2025-01-02 15:42:14,Suspensa,0,Aluno Novo,Sim,desistiu,Trabalho,desistiu,NaN
6,Four Corners A1 - 1S2025 - Capão Redondo - Fee...,Four Corners A1,4,Estudante 4 - Capão Redondo Four Corners A1,NaN,P,NaN,NaN,NaN,√,...,NaN,2025-05-06 18:05:45,Ativa,0,Aluno Novo,Sim,NaN,NaN,Aprovado,NaN


# Correção e normalização dos nomes das colunas

In [ ]:
def is_numeric_label(label):
    return bool(re.fullmatch(r"\d+", str(label).strip()))

# cria cópia dos nomes
new_cols = list(base.columns)

# percorre colunas
for i, col in enumerate(base.columns):
    if is_numeric_label(col):
        # pega o valor da linha 0 nessa coluna
        val = base.iloc[0, i]
        # se o valor for vazio/nan, ignora
        if pd.notna(val) and str(val).strip():
            new_cols[i] = str(val).strip()

# aplica os novos nomes
base.columns = new_cols

print("✅ Colunas renomeadas quando o header era numérico.")
for i, c in enumerate(base.columns[:20]):
    print(f"{i:>2}: {c}")


✅ Colunas renomeadas quando o header era numérico.
 0: Nome Planilha Feedback
 1: Sala
 2: Num
 3: NOME COMPLETO
 4: Aula 1 - 8-fev.-2025 - Pre-Class
 5: Aula 1 - 8-fev.-2025 - P
 6: Aula 1 - 8-fev.-2025 - Hw
 7: Aula 1 - 8-fev.-2025 - CP
 8: Aula 1 - 8-fev.-2025 - Bh
 9: Aula 2 - 15-fev.-2025 - Pre-Class
10: Aula 2 - 15-fev.-2025 - P
11: Aula 2 - 15-fev.-2025 - Hw
12: Aula 2 - 15-fev.-2025 - CP
13: Aula 2 - 15-fev.-2025 - Bh
14: Aula 3 - 22-fev.-2025 - Pre-Class
15: Aula 3 - 22-fev.-2025 - P
16: Aula 3 - 22-fev.-2025 - Hw
17: Aula 3 - 22-fev.-2025 - CP
18: Aula 3 - 22-fev.-2025 - Bh
19: Aula 4 - 8-mar.-2025 - Pre-Class


In [ ]:
base.head()

,Nome Planilha Feedback,Sala,Num,NOME COMPLETO,Aula 1 - 8-fev.-2025 - Pre-Class,Aula 1 - 8-fev.-2025 - P,Aula 1 - 8-fev.-2025 - Hw,Aula 1 - 8-fev.-2025 - CP,Aula 1 - 8-fev.-2025 - Bh,Aula 2 - 15-fev.-2025 - Pre-Class,...,e-mail Responsável,Data do Ultimo Login Conta CPM,Status da Conta CPM,Quantidade de reprovações no curso (4 reprovações = perda de vaga),Status aluno semestre anterior,Aluno pode trancar matrícula ou manter trancamento? (Alunos com status NÃO já fizeram trancamento no passado),Situação aluno no semestre,"Se aluno não está mais frequentando as aulas, escolha o motivo",Situação Final,Recomendação Conselho (Aprovado/Reprovado)
2,Nome Planilha Feedback,Sala,Num,NOME COMPLETO,Pre-Class,P,Hw,CP,Bh,Pre-Class,...,e-mail Responsável,Data do Ultimo Login Conta CPM,Status da Conta CPM,Quantidade de reprovações no curso (4 reprovaç...,Status aluno semestre anterior,Aluno pode trancar matrícula ou manter trancam...,Situação aluno no semestre,"Se aluno não está mais frequentando as aulas, ...",Situação Final,Recomendação Conselho (Aprovado/Reprovado)
3,Four Corners A1 - 1S2025 - Capão Redondo - Fee...,Four Corners A1,1,Estudante 1 - Capão Redondo Four Corners A1,NaN,A,NaN,NaN,NaN,√,...,NaN,2025-05-06 00:39:06,Ativa,0,Aluno Novo,Sim,NaN,NaN,Reprovado,NaN
4,Four Corners A1 - 1S2025 - Capão Redondo - Fee...,Four Corners A1,2,Estudante 2 - Capão Redondo Four Corners A1,NaN,A,NaN,NaN,NaN,√,...,NaN,2025-07-06 01:14:04,Ativa,0,Aluno Novo,Sim,NaN,NaN,Aprovado,NaN
5,Four Corners A1 - 1S2025 - Capão Redondo - Fee...,Four Corners A1,3,Estudante 3 - Capão Redondo Four Corners A1,NaN,A,NaN,NaN,NaN,N,...,NaN,2025-01-02 15:42:14,Suspensa,0,Aluno Novo,Sim,desistiu,Trabalho,desistiu,NaN
6,Four Corners A1 - 1S2025 - Capão Redondo - Fee...,Four Corners A1,4,Estudante 4 - Capão Redondo Four Corners A1,NaN,P,NaN,NaN,NaN,√,...,NaN,2025-05-06 18:05:45,Ativa,0,Aluno Novo,Sim,NaN,NaN,Aprovado,NaN


# Tratando os valores das colunas de aulas

In [ ]:
for col in base.columns:
    if isinstance(col, str) and col.endswith(' - P'):
        base.loc[:, col] = base[col].apply(lambda x: 1 if x == 'P' else (0 if x in ['A', '1/2'] else 0))

for col in base.columns:
    if isinstance(col, str) and col.endswith(' - Pre-Class'):
        base.loc[:, col] = base[col].apply(lambda x: 1 if x == '√' else (0 if x == 'N' else 0))

display(base.head())

,Nome Planilha Feedback,Sala,Num,NOME COMPLETO,Aula 1 - 8-fev.-2025 - Pre-Class,Aula 1 - 8-fev.-2025 - P,Aula 1 - 8-fev.-2025 - Hw,Aula 1 - 8-fev.-2025 - CP,Aula 1 - 8-fev.-2025 - Bh,Aula 2 - 15-fev.-2025 - Pre-Class,...,e-mail Responsável,Data do Ultimo Login Conta CPM,Status da Conta CPM,Quantidade de reprovações no curso (4 reprovações = perda de vaga),Status aluno semestre anterior,Aluno pode trancar matrícula ou manter trancamento? (Alunos com status NÃO já fizeram trancamento no passado),Situação aluno no semestre,"Se aluno não está mais frequentando as aulas, escolha o motivo",Situação Final,Recomendação Conselho (Aprovado/Reprovado)
2,Nome Planilha Feedback,Sala,Num,NOME COMPLETO,0,1,Hw,CP,Bh,0,...,e-mail Responsável,Data do Ultimo Login Conta CPM,Status da Conta CPM,Quantidade de reprovações no curso (4 reprovaç...,Status aluno semestre anterior,Aluno pode trancar matrícula ou manter trancam...,Situação aluno no semestre,"Se aluno não está mais frequentando as aulas, ...",Situação Final,Recomendação Conselho (Aprovado/Reprovado)
3,Four Corners A1 - 1S2025 - Capão Redondo - Fee...,Four Corners A1,1,Estudante 1 - Capão Redondo Four Corners A1,0,0,NaN,NaN,NaN,1,...,NaN,2025-05-06 00:39:06,Ativa,0,Aluno Novo,Sim,NaN,NaN,Reprovado,NaN
4,Four Corners A1 - 1S2025 - Capão Redondo - Fee...,Four Corners A1,2,Estudante 2 - Capão Redondo Four Corners A1,0,0,NaN,NaN,NaN,1,...,NaN,2025-07-06 01:14:04,Ativa,0,Aluno Novo,Sim,NaN,NaN,Aprovado,NaN
5,Four Corners A1 - 1S2025 - Capão Redondo - Fee...,Four Corners A1,3,Estudante 3 - Capão Redondo Four Corners A1,0,0,NaN,NaN,NaN,0,...,NaN,2025-01-02 15:42:14,Suspensa,0,Aluno Novo,Sim,desistiu,Trabalho,desistiu,NaN
6,Four Corners A1 - 1S2025 - Capão Redondo - Fee...,Four Corners A1,4,Estudante 4 - Capão Redondo Four Corners A1,0,1,NaN,NaN,NaN,1,...,NaN,2025-05-06 18:05:45,Ativa,0,Aluno Novo,Sim,NaN,NaN,Aprovado,NaN


In [ ]:
for col in base.columns:
    if isinstance(col, str) and col.endswith(' - Hw'):
        base.loc[:, col] = base[col].apply(lambda x: 0 if pd.isna(x) else 1)


for col in base.columns:
    if isinstance(col, str) and col.endswith(' - CP'):
        base.loc[:, col] = base[col].apply(lambda x: 2 if x == ':-D' else (1 if x == ':-/' else (0 if x == ':-&' else -1)))


for col in base.columns:
    if isinstance(col, str) and col.endswith(' - Bh'):
        base.loc[:, col] = base[col].apply(lambda x: 1 if x == ':-(' else 0)


display(base.head())

,Nome Planilha Feedback,Sala,Num,NOME COMPLETO,Aula 1 - 8-fev.-2025 - Pre-Class,Aula 1 - 8-fev.-2025 - P,Aula 1 - 8-fev.-2025 - Hw,Aula 1 - 8-fev.-2025 - CP,Aula 1 - 8-fev.-2025 - Bh,Aula 2 - 15-fev.-2025 - Pre-Class,...,e-mail Responsável,Data do Ultimo Login Conta CPM,Status da Conta CPM,Quantidade de reprovações no curso (4 reprovações = perda de vaga),Status aluno semestre anterior,Aluno pode trancar matrícula ou manter trancamento? (Alunos com status NÃO já fizeram trancamento no passado),Situação aluno no semestre,"Se aluno não está mais frequentando as aulas, escolha o motivo",Situação Final,Recomendação Conselho (Aprovado/Reprovado)
2,Nome Planilha Feedback,Sala,Num,NOME COMPLETO,0,1,1,-1,0,0,...,e-mail Responsável,Data do Ultimo Login Conta CPM,Status da Conta CPM,Quantidade de reprovações no curso (4 reprovaç...,Status aluno semestre anterior,Aluno pode trancar matrícula ou manter trancam...,Situação aluno no semestre,"Se aluno não está mais frequentando as aulas, ...",Situação Final,Recomendação Conselho (Aprovado/Reprovado)
3,Four Corners A1 - 1S2025 - Capão Redondo - Fee...,Four Corners A1,1,Estudante 1 - Capão Redondo Four Corners A1,0,0,0,-1,0,1,...,NaN,2025-05-06 00:39:06,Ativa,0,Aluno Novo,Sim,NaN,NaN,Reprovado,NaN
4,Four Corners A1 - 1S2025 - Capão Redondo - Fee...,Four Corners A1,2,Estudante 2 - Capão Redondo Four Corners A1,0,0,0,-1,0,1,...,NaN,2025-07-06 01:14:04,Ativa,0,Aluno Novo,Sim,NaN,NaN,Aprovado,NaN
5,Four Corners A1 - 1S2025 - Capão Redondo - Fee...,Four Corners A1,3,Estudante 3 - Capão Redondo Four Corners A1,0,0,0,-1,0,0,...,NaN,2025-01-02 15:42:14,Suspensa,0,Aluno Novo,Sim,desistiu,Trabalho,desistiu,NaN
6,Four Corners A1 - 1S2025 - Capão Redondo - Fee...,Four Corners A1,4,Estudante 4 - Capão Redondo Four Corners A1,0,1,0,-1,0,1,...,NaN,2025-05-06 18:05:45,Ativa,0,Aluno Novo,Sim,NaN,NaN,Aprovado,NaN


# Dicts e funções para renomear colunas numéricas com valores da primeira linha

In [ ]:
PT_MONTH_MAP = {
    "jan": "jan", "fev": "fev", "mar": "mar", "abr": "abr", "mai": "mai", "jun": "jun",
    "jul": "jul", "ago": "ago", "set": "set", "out": "out", "nov": "nov", "dez": "dez"
}

COMP_MAP = {
    "pre-class": "pre_class",
    "pre class": "pre_class",
    "p": "p",
    "hw": "hw",
    "cp": "cp",
    "bh": "bh",
}

def strip_accents(s: str) -> str:
    s = unicodedata.normalize("NFKD", s)
    return "".join(c for c in s if not unicodedata.combining(c))

def slugify(s: str) -> str:
    s = strip_accents(s or "")
    s = s.lower().strip()
    s = s.replace("%", " percent ")
    s = re.sub(r"[^0-9a-z_]+", "_", s)
    s = re.sub(r"_+", "_", s).strip("_")
    return s or "coluna"

def normalize_date_token(dt: str) -> str:
    """Converte '8-fev.-2025' -> '8_fev_2025'; aceita variações com ponto/traços/espaços."""
    if not dt:
        return "data_indefinida"
    t = strip_accents(str(dt).lower())
    t = re.sub(r"[^\w\-\. ]+", "", t)
    parts = re.split(r"[-\.\s]+", t)
    parts = [p for p in parts if p]
    if len(parts) >= 3:
        d, m, y = parts[0], parts[1], parts[2]
        m = m[:3]
        m = PT_MONTH_MAP.get(m, m)
        return f"{d}_{m}_{y}"
    return slugify(t)

def make_unique(names):
    seen = {}
    out = []
    for n in names:
        if n not in seen:
            seen[n] = 0
            out.append(n)
        else:
            seen[n] += 1
            out.append(f"{n}__{seen[n]}")
    return out

AULA_RE = re.compile(
    r"^\s*Aula\s*(\d+)\s*-\s*([^-]+?)\s*-\s*([A-Za-z\- ]+)\s*$",
    flags=re.IGNORECASE
)

def map_aula_col(name: str) -> str | None:
    m = AULA_RE.match(str(name))
    if not m:
        return None
    aula_num = m.group(1)
    dt_raw = m.group(2)
    comp_raw = m.group(3)

    dt_norm = normalize_date_token(dt_raw)
    comp_key = slugify(comp_raw).replace("-", "_").replace("__", "_")
    comp_key = COMP_MAP.get(comp_key, comp_key)
    return f"aula_{aula_num}_{dt_norm}_{comp_key}"

SPECIAL_MAP = {
    "Nome Planilha Feedback": "nome_planilha_feedback",
    "Sala": "sala",
    "Num": "num",
    "NOME COMPLETO": "nome_completo",
    "Media Provas": "media_provas",
    "Nota Final": "nota_final",
    "Situação Final": "situacao_final",
    "Situação aluno no semestre": "situacao_aluno_no_semestre",
    "Se aluno não está mais frequentando as aulas, escolha o motivo": "motivo_nao_frequentando",
    "Recomendação Conselho (Aprovado/Reprovado)": "recomendacao_conselho_aprovado_slash_reprovado",
    "Status": "status",
    "Data de nascimento": "data_de_nascimento",
    "Telefone Celular": "telefone_celular",
    "Telefone Fixo": "telefone_fixo",
    "e-mail CPM": "e_mail_cpm",
    "e-mail Pessoal": "e_mail_pessoal",
    "Nome Completo Responsável": "nome_completo_responsavel",
    "Celular Responsável": "celular_responsavel",
    "e-mail Responsável": "e_mail_responsavel",
    "Data do Ultimo Login Conta CPM": "data_do_ultimo_login_conta_cpm",
    "Status da Conta CPM": "status_da_conta_cpm",
    "Quantidade de reprovações no curso (4 reprovações = perda de vaga)": "quantidade_de_reprovacoes_no_curso_4_reprovacoes_perda_de_vaga",
    "Status aluno semestre anterior": "status_aluno_semestre_anterior",
}

PREF_OK = ("pre_class_", "presence_", "homework_", "class_participation_", "behavior_")

✅ Renomeação concluída. Arquivo de-para salvo em depara_renomeacao.csv
Exemplo das primeiras 25 colunas:
Index(['nome_planilha_feedback', 'sala', 'num', 'nome_completo',
       'aula_1_8_fev_2025_pre_class', 'aula_1_8_fev_2025_p',
       'aula_1_8_fev_2025_hw', 'aula_1_8_fev_2025_cp', 'aula_1_8_fev_2025_bh',
       'aula_2_15_fev_2025_pre_class', 'aula_2_15_fev_2025_p',
       'aula_2_15_fev_2025_hw', 'aula_2_15_fev_2025_cp',
       'aula_2_15_fev_2025_bh', 'aula_3_22_fev_2025_pre_class',
       'aula_3_22_fev_2025_p', 'aula_3_22_fev_2025_hw',
       'aula_3_22_fev_2025_cp', 'aula_3_22_fev_2025_bh',
       'aula_4_8_mar_2025_pre_class', 'aula_4_8_mar_2025_p',
       'aula_4_8_mar_2025_hw', 'aula_4_8_mar_2025_cp', 'aula_4_8_mar_2025_bh',
       'aula_5_15_mar_2025_pre_class'],
      dtype='object')


In [ ]:
old_cols = list(base.columns)
new_cols = []

for col in old_cols:
    col_str = str(col)

    aula_norm = map_aula_col(col_str)
    if aula_norm:
        new_cols.append(aula_norm)
        continue

    if col_str in SPECIAL_MAP:
        new_cols.append(SPECIAL_MAP[col_str])
        continue

    if any(col_str.startswith(p) for p in PREF_OK):
        new_cols.append(col_str)
        continue

    new_cols.append(slugify(col_str))

new_cols = make_unique(new_cols)
base.columns = new_cols
depara = pd.DataFrame({"col_original": old_cols, "col_nova": new_cols})
depara.to_csv("depara_renomeacao.csv", index=False)

print("✅ Renomeação concluída. Arquivo de-para salvo em depara_renomeacao.csv")
print("Exemplo das primeiras 25 colunas:")
print(base.columns[:25])

In [ ]:
base = base.iloc[1:].reset_index().drop(columns=['index'])

# Tratamento de dummies e valores não numéricos das colunas

In [ ]:
SENTINEL_FLOAT = -9999.0

def to_float_br(x):
    if pd.isna(x): return np.nan
    t = str(x).strip()
    if t == "" or t.lower() in {"nan","none","null","na","n/a","-","--"}:
        return np.nan
    is_pct = t.endswith("%")
    if is_pct: t = t[:-1].strip()
    t = t.replace(" ", "")
    if "," in t:
        if "." not in t:
            t = t.replace(",", ".")
        else:
            t = t.replace(".", "").replace(",", ".")
    t = re.sub(r"[^0-9eE+\-\.]", "", t)
    if t in ("", "+", "-", ".", "+.", "-."): return np.nan
    try:
        v = float(t)
        return v/100.0 if is_pct else v
    except Exception:
        return np.nan

START_COL = "aula_14_data_indefinida_bh"

EXCEPT_COLS = {
    "data_de_nascimento",
    "telefone_celular",
    "telefone_fixo",
    "e_mail_cpm",
    "e_mail_pessoal",
    "nome_completo_responsavel",
    "celular_responsavel",
    "e_mail_responsavel",
    "data_do_ultimo_login_conta_cpm",
    "status_da_conta_cpm",
}

if START_COL not in base.columns:
    raise KeyError(f"Não encontrei a coluna '{START_COL}' nas colunas atuais.")

start_idx = base.columns.get_loc(START_COL)

target_cols = [c for c in base.columns[start_idx:] if c not in EXCEPT_COLS]

convertidas, falhas = [], []

for c in target_cols:
    try:
        v = base_tratada[c].map(to_float_br).astype("float64")
        base_tratada[c] = v.fillna(SENTINEL_FLOAT)
        convertidas.append(c)
    except Exception as e:
        falhas.append((c, str(e)))

print(f"✅ Convertidas para float com sentinela ({SENTINEL_FLOAT}): {len(convertidas)} colunas")
if falhas:
    print(f"⚠️ Não convertidas ({len(falhas)}): " + ", ".join([f[0] for f in falhas]))

base_tratada

✅ Convertidas para float com sentinela (-9999.0): 45 colunas


,nome_planilha_feedback,sala,num,nome_completo,aula_1_8_fev_2025_pre_class,aula_1_8_fev_2025_p,aula_1_8_fev_2025_hw,aula_1_8_fev_2025_cp,aula_1_8_fev_2025_bh,aula_2_15_fev_2025_pre_class,...,e_mail_responsavel,data_do_ultimo_login_conta_cpm,status_da_conta_cpm,quantidade_de_reprovacoes_no_curso_4_reprovacoes_perda_de_vaga,status_aluno_semestre_anterior,aluno_pode_trancar_matricula_ou_manter_trancamento_alunos_com_status_nao_ja_fizeram_trancamento_no_passado,situacao_aluno_no_semestre,motivo_nao_frequentando,situacao_final__1,recomendacao_conselho_aprovado_slash_reprovado
0,Four Corners A1 - 1S2025 - Capão Redondo - Fee...,Four Corners A1,1,Estudante 1 - Capão Redondo Four Corners A1,0,0,0,-1,0,1,...,NaN,2025-05-06 00:39:06,Ativa,0.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0
1,Four Corners A1 - 1S2025 - Capão Redondo - Fee...,Four Corners A1,2,Estudante 2 - Capão Redondo Four Corners A1,0,0,0,-1,0,1,...,NaN,2025-07-06 01:14:04,Ativa,0.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0
2,Four Corners A1 - 1S2025 - Capão Redondo - Fee...,Four Corners A1,3,Estudante 3 - Capão Redondo Four Corners A1,0,0,0,-1,0,0,...,NaN,2025-01-02 15:42:14,Suspensa,0.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0
3,Four Corners A1 - 1S2025 - Capão Redondo - Fee...,Four Corners A1,4,Estudante 4 - Capão Redondo Four Corners A1,0,1,0,-1,0,1,...,NaN,2025-05-06 18:05:45,Ativa,0.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0
4,Four Corners A1 - 1S2025 - Capão Redondo - Fee...,Four Corners A1,5,Estudante 5 - Capão Redondo Four Corners A1,0,1,0,-1,0,1,...,NaN,2025-06-06 18:18:41,Ativa,0.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1972,Four Corners J1 - 1S2025 - Unidade Virtual (au...,Four Corners J1,15,Estudante 15 - Unidade Virtual (aulas aos domi...,0,0,0,-1,0,0,...,NaN,2025-06-07 10:38:46,Ativa,1.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0
1973,Four Corners J1 - 1S2025 - Unidade Virtual (au...,Four Corners J1,16,Estudante 16 - Unidade Virtual (aulas aos domi...,0,1,0,-1,0,0,...,NaN,2025-06-15 14:02:23,Ativa,2.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0
1974,Four Corners J1 - 1S2025 - Unidade Virtual (au...,Four Corners J1,17,Estudante 17 - Unidade Virtual (aulas aos domi...,0,1,0,-1,0,0,...,NaN,2025-11-07 09:36:19,Ativa,2.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0
1975,Four Corners J1 - 1S2025 - Unidade Virtual (au...,Four Corners J1,18,Estudante 18 - Unidade Virtual (aulas aos domi...,0,0,0,-1,0,0,...,NaN,2025-07-07 13:09:47,Ativa,0.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0
